In [2]:


import os
from PIL import Image
import cv2
import numpy as np
def extract_word(string):
    start_index = string.find('_') + 1  # Find the index of the underscore and add 1 to exclude it
    end_index = string.find('.')  # Find the index of the dot

    if start_index < end_index and start_index > 0:
        return string[start_index:end_index]
    else:
        return None
def extract_name(string):
#     start_index = string.find('_') - 3  # Find the index of the underscore and add 1 to exclude it
    end_index = string.find('_')  # Find the index of the dot

    if 0 < end_index :
        return string[0:end_index]
    else:
        return None
def load_dataset(folder_path):
    dataset = []
    labels = []
    people = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Add more extensions if needed
                image_path = os.path.join(root, file)
                label = os.path.basename(file)
                name = extract_name(label)
                label = extract_word(label)
                
                try:
                    image = Image.open(image_path) 
                    image = cv2.resize(np.array(image,dtype=np.float32),(150,150)) / 255.0# Use PIL to open the image
                    dataset.append(np.squeeze(image))
                    labels.append(label[0])
                    people.append(name)
                except (IOError, OSError):
                    print(f"Error loading image: {image_path}")

    return dataset, labels, people

# Example usage
data_path = 'RADIATE_JPEGS1'
dataset_folder = data_path
images, labels, people = load_dataset(dataset_folder)

# Access individual images and labels
for image, label, person in zip(images, labels,people):
    # image.show()  # Display the image using PIL
    print(label)
#count of all the labels 
uniquelabels =[]
countlabels = []
id = 0

for i in labels:
  if i not in uniquelabels:
    uniquelabels.append(i)
for i in uniquelabels:
  count = 0
  for j in labels:
    if i == j:
      count += 1
  countlabels.append(count)
  id +=1
print(uniquelabels)
print(countlabels)
print(sum(countlabels))

truelabels = np.zeros(len(labels))
for tag,i in enumerate(labels):
    for id,j in enumerate(uniquelabels):
        if i == j:
            truelabels[tag]= id
#             print(tag)
            

C
C
S
F
H
S
N
S
A
H
D
D
A
H
F
N
S
S
S
H
A
N
F
H
H
D
C
N
A
D
C
F
H
N
N
S
F
S
D
S
A
H
C
F
C
A
D
H
S
H
D
C
C
A
N
S
F
F
A
N
H
D
S
C
H
F
H
H
A
A
D
S
H
H
F
D
A
S
C
D
N
F
H
F
S
H
D
H
C
S
N
H
D
S
S
S
F
N
A
C
A
N
D
H
H
C
F
A
C
F
N
A
C
F
S
S
H
N
N
F
C
S
A
C
D
F
H
S
H
H
A
N
S
D
D
S
S
A
D
S
N
C
S
C
N
A
F
S
F
D
H
H
H
A
C
N
C
A
F
H
D
H
N
F
C
S
H
S
N
A
F
H
S
A
D
C
A
D
H
C
H
F
A
H
S
S
C
D
F
N
S
C
S
F
H
D
H
A
A
H
N
A
S
H
S
N
D
S
C
D
H
N
F
C
S
H
S
N
A
F
H
S
A
D
C
A
C
S
S
H
S
A
C
S
N
C
S
H
D
H
D
A
N
F
H
F
S
H
F
C
C
D
N
N
S
A
A
D
F
H
H
S
N
C
A
D
A
N
F
C
S
A
C
D
F
H
S
H
H
A
N
S
D
H
D
S
S
S
F
N
A
C
A
N
D
H
H
C
F
N
D
F
S
C
D
S
S
A
H
F
H
A
C
N
H
H
A
H
S
D
C
S
N
F
H
F
C
D
N
S
A
S
H
A
F
H
N
S
C
A
S
N
H
H
F
S
D
D
C
A
F
D
F
A
D
S
S
H
H
H
C
N
N
S
C
F
C
A
N
D
F
S
A
S
D
S
H
H
H
C
N
H
N
D
F
H
S
A
C
H
S
S
A
F
D
C
N
H
C
N
H
F
F
S
A
D
N
D
A
C
H
S
S
S
D
A
S
A
D
N
H
N
A
S
D
C
N
S
S
H
F
F
C
H
D
A
C
S
F
C
H
H
N
F
D
D
S
H
S
A
N
A
H
H
N
S
N
F
A
D
A
C
S
D
S
H
F
C
D
H
H
A
C
F
H
S
A
N
N
S
S
C
F
D
C
H
H
S
C
F
A
D
H
A
N
S
D
F
N
S


In [80]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Specify which GPU to use, e.g., "0" for GPU 0

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load the MediaPipe face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=0.1)

# Custom Dataset class with MediaPipe feature extraction
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_images = self.images[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

        # Extract features using MediaPipe for each image
        batch_features = [extract_features_from_image(img) for img in batch_images]
        batch_features = np.array(batch_features)
        batch_features = tf.reshape(batch_features, (81, 468, 3, 1))
        return batch_features, np.array(batch_labels)

# Function to extract features using MediaPipe from each image
def extract_features_from_image(image):
    # Convert image to RGB format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert image to uint8 format
    image_uint8 = np.clip(image_rgb * 255.0, 0, 255).astype(np.uint8)

    # Process the image with MediaPipe
    results = face_mesh.process(image_uint8)

    # Get the facial landmarks for the first face (if any)
    if results.multi_face_landmarks:
        facial_landmarks = results.multi_face_landmarks[0]
        # Process the facial landmarks (example only, replace with relevant features for your task)
        features = []
        for landmark in facial_landmarks.landmark:
            features.extend([landmark.x, landmark.y, landmark.z])
        return features
    else:
        # Return None if no face landmarks are detected
        return None
 # Replace with your label data

# Normalize and reshape the images
# images = cv2.resize(np.array(images),(150,150))
# images = np.expand_dims(images, axis=-1)

# Convert labels to categorical format
labels = np.array(truelabels)
labels = to_categorical(labels)
labels = tf.one_hot(labels, depth=7)
labels = tf.reshape(labels, (-1,))  # Reshape labels to have shape (batch_size,)



# Split the data into training and validation sets
import numpy as np
import torch
from torch.utils.data import DataLoader

# Split the data into train, validation, and test sets
split_train = int(0.6 * len(images))
split_val = int(0.8 * len(images))

train_images, train_labels = images[:split_train], labels[:split_train]
val_images, val_labels = images[split_train:split_val], labels[split_train:split_val]
test_images, test_labels = images[split_val:], labels[split_val:]

# Define batch size for train, validation, and test sets
train_batch_size = 81
val_batch_size = 81
test_batch_size = 81

# Create custom datasets for train, validation, and test
train_dataset = CustomDataset(train_images, train_labels, train_batch_size)
val_dataset = CustomDataset(val_images, val_labels, val_batch_size)
test_dataset = CustomDataset(test_images, test_labels, test_batch_size)


In [74]:
import tensorflow as tf

class LargerCNNWithAttention(tf.keras.Model):
    def __init__(self):
        super(LargerCNNWithAttention, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', activation='relu')
        self.attention1 = tf.keras.layers.Conv2D(1, kernel_size=1, strides=1, activation='sigmoid')
        self.conv4 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', activation='relu')
        self.attention2 = tf.keras.layers.Conv2D(1, kernel_size=1, strides=1, activation='sigmoid')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(1024, activation='relu')
        self.attention3 = tf.keras.layers.Dense(1, activation='sigmoid')
        self.fc2 = tf.keras.layers.Dense(7)  # Assuming 7 output classes

    def call(self, inputs):
        x = self.conv1(inputs)
        attention_map1 = self.attention1(x)
        x = tf.multiply(x, attention_map1)
        x = tf.keras.layers.MaxPooling2D(2)(x)

        x = self.conv2(x)
        attention_map2 = self.attention2(x)
        x = tf.multiply(x, attention_map2)
        x = tf.keras.layers.MaxPooling2D(2)(x)

        x = self.conv3(x)
        x = tf.keras.layers.MaxPooling2D(2)(x)

        x = self.conv4(x)
        x = tf.keras.layers.MaxPooling2D(2)(x)

        x = self.flatten(x)
        x = self.fc1(x)
        attention_weights = self.attention3(x)
        x = tf.multiply(x, attention_weights)

        x = self.fc2(x)
        return x

# Initialize the larger model with attention
model = LargerCNNWithAttention()


In [75]:
# Define loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
batch_size = 100
epochs = 10
model.fit(train_dataset, batch_size=batch_size, epochs=epochs, validation_data=(val_dataset))



Epoch 1/10
9/9 [==============================] - 7s 770ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 2/10
9/9 [==============================] - 8s 833ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 3/10
9/9 [==============================] - 6s 684ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 4/10
9/9 [==============================] - 7s 753ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 5/10
9/9 [==============================] - 7s 767ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 6/10
9/9 [==============================] - 7s 709ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 7/10
9/9 [==============================] - 7s 835ms/step - loss: 1.9459 - accuracy: 0.8560 - val_loss: 1.9459 - val_accuracy: 0.8601
Epoch 8/10
9/9 [====

In [86]:
for id,(i, j) in enumerate(test_dataset):
    if id <2:
        model.evaluate(i,j)

3/3 [==============================] - 0s 4ms/step - loss: 1.9459 - accuracy: 0.8642


/tmp/ipykernel_125905/1598755323.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_features = np.array(batch_features)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [83]:
test_new = []
for id, (i,j) in enumerate(test_dataset):
    print(np.array(i).shape,id,np.array(j).shape)
    test_new.append(i,j)

(81, 468, 3, 1) 0 (81,)


TypeError: list.append() takes exactly one argument (2 given)

In [72]:
for id, (i,j) in enumerate(train_dataset):
    print(np.array(i).shape,id,np.array(j).shape)

(81, 1404) 0 (81,)
(81, 1404) 1 (81,)
(81, 1404) 2 (81,)
(81, 1404) 3 (81,)
(81, 1404) 4 (81,)
(81, 1404) 5 (81,)
(81, 1404) 6 (81,)
(81, 1404) 7 (81,)
(81, 1404) 8 (81,)


In [88]:
pred = []
for i,j in test_dataset:
    print(pred.append(model.predict(i)))

ValueError: in user code:

    File "/home/lunet/conn3/.local/lib/python3.9/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/lunet/conn3/.local/lib/python3.9/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/lunet/conn3/.local/lib/python3.9/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/lunet/conn3/.local/lib/python3.9/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/lunet/conn3/.local/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_fileaaat9_ml.py", line 17, in tf__call
        x = ag__.conve

In [21]:
model.summary()

Model: "larger_cnn_with_attention_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          multiple                  0 (unused)
                                                                 
 conv2d_23 (Conv2D)          multiple                  0 (unused)
                                                                 
 conv2d_24 (Conv2D)          multiple                  0 (unused)
                                                                 
 conv2d_25 (Conv2D)          multiple                  0 (unused)
                                                                 
 conv2d_26 (Conv2D)          multiple                  0 (unused)
                                                                 
 conv2d_27 (Conv2D)          multiple                  0 (unused)
                                                                 
 flatten_5 (Flatten)         multiple  